In [151]:
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd
from PIL import Image

plt.style.use('dark_background')

In [195]:
def scout_power(server):
    power = list()

    for i in range(50):
        initial = Image.open('./data/' + server + '/original/' + str(i+1) + '.png')
        width = initial.size[0]
        height = initial.size[1]

        croped = initial.crop((width*0.72, height*0.27, width*0.85, height*0.92))
        croped.save('./data/' + server + '/power/' + str(i+1) + '.png')

        img_ori = cv2.imread('data/' + server + '/power/' + str(i+1) + '.png')

        gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
        structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

        imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
        imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

        imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
        gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

        img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

        ret, img_thresh = cv2.threshold(
            img_blurred, 
            180,
            255,
            cv2.THRESH_BINARY_INV
        )

    #     plt.figure(figsize=(12, 10))
    #     plt.imshow(img_thresh, cmap='gray')

        chars = pytesseract.image_to_string(img_thresh)

        temp_power = chars.split('\n')
        temp_power = list(filter(None, temp_power))

        for tmp in temp_power:
            list_of_numbers = re.findall(r'\d+', tmp)
            result_number = ''.join(list_of_numbers)
            power.append(int(result_number))
            
    df_power = pd.DataFrame({server + ' POWER':power})
    pd.options.display.float_format = '{:.5f}'.format
    df_describe = df_power.describe()
    
    result = pd.ExcelWriter('data/' + server + '/result.xlsx', engine='xlsxwriter')
    sh_name1 = server + ' power'
    sh_name2 = server + ' describe'
    df_power.to_excel(result, sheet_name=sh_name1)
    df_describe.to_excel(result, sheet_name=sh_name2)

    result.save()

In [196]:
scout_power('1719')